# "Binarized Neural Networks: Training Deep Neural Networks with Weights and Activations Constrained to +1 or -1" paper implementation - https://arxiv.org/abs/1602.02830

In [1]:
import torch
import torch.nn as nn

def binarize(tensor,quant_mode='det'):
    if quant_mode=='det':
        return tensor.sign()
    else:
        return tensor.add_(1).div_(2).add_(torch.rand(tensor.size()).add(-0.5)).clamp_(0,1).round().mul_(2).add_(-1)

class BinarizeLinear(nn.Linear):
    def __init__(self, *kargs, **kwargs):
        super(BinarizeLinear, self).__init__(*kargs, **kwargs)

    def forward(self, input):
        if input.size(1) != 784:
            input.data = binarize(input.data)
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
        self.weight.data = binarize(self.weight.org)
        out = nn.functional.linear(input, self.weight)
        if not self.bias is None:
            self.bias.org=self.bias.data.clone()
            out += self.bias.view(1, -1).expand_as(out)
        return out

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.infl_ratio = 3
        self.model = nn.Sequential(
            BinarizeLinear(784, 2048 * self.infl_ratio),
            nn.Hardtanh(),
            nn.BatchNorm1d(2048 * self.infl_ratio),
            BinarizeLinear(2048 * self.infl_ratio, 2048 * self.infl_ratio),
            nn.Hardtanh(),
            nn.BatchNorm1d(2048 * self.infl_ratio),
            BinarizeLinear(2048 * self.infl_ratio, 2048 * self.infl_ratio),
            nn.Hardtanh(),
            nn.BatchNorm1d(2048 * self.infl_ratio),
            nn.Linear(2048 * self.infl_ratio, 10),
            nn.LogSoftmax(),
            nn.Dropout(0.5)
        )

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        return self.model(x)
    
model = Net()

In [8]:
from torchvision import datasets, transforms

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=64, shuffle=True)


In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)

        if epoch % 40 == 0:
            optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.1

        optimizer.zero_grad()
        loss.backward()
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.data.copy_(p.org)
        optimizer.step()
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.org.copy_(p.data.clamp_(-1,1))

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 20 + 1):
    train(epoch)
    test()
    if epoch == 10:
        optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.1

/home/petko/projects/ai-notebooks/env/lib/python3.11/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 1 [0/60000 (0%)]	Loss: 3.973456
Train Epoch: 1 [640/60000 (1%)]	Loss: 3.791637
Train Epoch: 1 [1280/60000 (2%)]	Loss: 4.272464
Train Epoch: 1 [1920/60000 (3%)]	Loss: 3.686195
Train Epoch: 1 [2560/60000 (4%)]	Loss: 3.870046
Train Epoch: 1 [3200/60000 (5%)]	Loss: 4.171551
Train Epoch: 1 [3840/60000 (6%)]	Loss: 3.677016
Train Epoch: 1 [4480/60000 (7%)]	Loss: 8.586338
Train Epoch: 1 [5120/60000 (9%)]	Loss: 3.912142
Train Epoch: 1 [5760/60000 (10%)]	Loss: 3.920439
Train Epoch: 1 [6400/60000 (11%)]	Loss: 3.885495
Train Epoch: 1 [7040/60000 (12%)]	Loss: 3.897737
Train Epoch: 1 [7680/60000 (13%)]	Loss: 4.098632
Train Epoch: 1 [8320/60000 (14%)]	Loss: 4.006138
Train Epoch: 1 [8960/60000 (15%)]	Loss: 3.764494
Train Epoch: 1 [9600/60000 (16%)]	Loss: 3.844500
Train Epoch: 1 [10240/60000 (17%)]	Loss: 3.729693
Train Epoch: 1 [10880/60000 (18%)]	Loss: 4.337928
Train Epoch: 1 [11520/60000 (19%)]	Loss: 3.858698
Train Epoch: 1 [12160/60000 (20%)]	Loss: 4.299174
Train Epoch: 1 [12800/60000 (